In [1]:
# imports
# standard libraries
import pandas as pd
import numpy as np

# statistics
from scipy import stats

# visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Options to display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:,.3f}'.format

In [3]:
df = pd.read_csv("../data/raw/Customer-Churn-Records.csv")

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.000,1,1,1,"101,348.880",1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,"83,807.860",1,0,1,"112,542.580",0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,"159,660.800",3,1,0,"113,931.570",1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.000,2,0,0,"93,826.630",0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,"125,510.820",1,1,1,"79,084.100",0,0,5,GOLD,425


In [5]:
# Drop rownumber because the column is not necessary
df.drop(columns={"RowNumber", "Surname", "Complain", "CustomerId"}, inplace=True)

In [ ]:
# Renaming columns for Consistency
df.rename(columns={
    'Satisfaction Score	': 'SatisfactionScore',
    'Card Type': 'CardType',
    'Point Earned': 'PointEarned'
}, inplace=True)

Outliers

In [7]:
# Subset the dataframe; only numeric columns
numeric_df = df.select_dtypes(include=['number'])

# IQR
Q1 = numeric_df.quantile(0.25)
Q3 = numeric_df.quantile(0.75)
IQR = Q3 - Q1

In [9]:
# Get numeric columns excluding Client_Id
columns_to_winsorize = numeric_df.columns[numeric_df.columns != 'CustomerId']

In [ ]:
# # Winsorization on outliers
for column in columns_to_winsorize:
    # Calculate lower and upper bounds
    lower_bound = Q1[column] - 1.5 * IQR[column]
    upper_bound = Q3[column] + 1.5 * IQR[column]
    
    # Apply winsorization
    numeric_df[column] = numeric_df[column].clip(lower=lower_bound, upper=upper_bound)

In [13]:
numeric_df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Satisfaction Score,PointEarned
0,619,42,2,0.000,1.000,1,1,"101,348.880",0,2,464
1,608,41,1,"83,807.860",1.000,0,1,"112,542.580",0,3,456
2,502,42,8,"159,660.800",3.000,1,0,"113,931.570",0,3,377
3,699,39,1,0.000,2.000,0,0,"93,826.630",0,5,350
4,850,43,2,"125,510.820",1.000,1,1,"79,084.100",0,5,425


In [15]:
# Dictionary to hold normality test results
normality_results = {}

for column in numeric_df.columns:
    # Perform the Shapiro-Wilk test
    stat, p_value = stats.shapiro(numeric_df[column].dropna())
    normality_results[column] = p_value > 0.05  # True if normal, False otherwise

normality_results
# DataFrame for better readability
#normality_df = pd.DataFrame.from_dict(normality_results, orient='index', columns=['Is_Normally_Distributed'])
#normality_df

{'CreditScore': False,
 'Age': False,
 'Tenure': False,
 'Balance': False,
 'NumOfProducts': False,
 'HasCrCard': False,
 'IsActiveMember': False,
 'EstimatedSalary': False,
 'Exited': True,
 'Satisfaction Score': False,
 'PointEarned': False}

### Categorical Feature Encodings

In [16]:
# Select categorical features
categorical_features = df.select_dtypes(include=['object']).columns
categorical_features

Index(['Geography', 'Gender', 'CardType'], dtype='object')

In [17]:
categorical_features = categorical_features[categorical_features != 'Surname']

In [18]:
df[categorical_features].head()

,Geography,Gender,CardType
0,France,Female,DIAMOND
1,Spain,Female,DIAMOND
2,France,Female,DIAMOND
3,France,Female,GOLD
4,Spain,Female,GOLD


In [19]:
for col in df[categorical_features]:
    print(f"Feature: {col}")
    print(df[col].value_counts())
    print("---\n")

Feature: Geography
Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64
---

Feature: Gender
Gender
Male      5457
Female    4543
Name: count, dtype: int64
---

Feature: CardType
CardType
DIAMOND     2507
GOLD        2502
SILVER      2496
PLATINUM    2495
Name: count, dtype: int64
---



In [20]:
# One-hot encode the 'Geography' column
df = pd.get_dummies(df, columns=['Geography'], prefix='Geography')

In [21]:
# One-hot encoding for 'Gender' column // Alternative: binary encoding
df = pd.get_dummies(df, columns=['Gender'], prefix='Gender')

In [22]:
# mapping for ordinal encoding for Card_Category
card_mapping = {
    'SILVER': 0,
    'GOLD': 1,
    'PLATINUM': 2,
    'DIAMOND': 3
}

df['CardType'] = df['CardType'].map(card_mapping)

In [24]:
df.to_csv('../data/cleaned/CustomerChurnCleaned.csv', index=False)